# 00 数据接入与事件分析准备

**研究背景**: Charlie Kirk政治暗杀事件社交媒体舆论分析  
**事件时间**: 2025-09-10 枪击发生  
**数据时间窗口**: 2025-09-11 23:55 至 2025-09-13 00:12 (事件后72小时)

**目标**:
1. 加载原始CSV数据并规范化类型
2. 合并作者元数据
3. **添加事件相关时间字段**（核心优化）
4. 写入Parquet缓存供后续分析

In [1]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
project_root = Path('/workspace')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
print(f"✅ Python 路径已配置: {project_root}")

✅ Python 路径已配置: /workspace


## 步骤 1: 加载原始数据

In [2]:
from src import io, analysis, profiling
import polars as pl
from datetime import datetime, timezone

# 加载原始推文数据 (LazyFrame)
raw_lf = io.scan_raw_tweets()
print(f"📊 数据 schema:")
print(raw_lf.collect_schema())

📊 数据 schema:
Schema([('pseudo_id', Int64), ('text', String), ('retweetCount', Int64), ('replyCount', Int64), ('likeCount', Int64), ('quoteCount', Int64), ('viewCount', Int64), ('bookmarkCount', Int64), ('createdAt', String), ('lang', String), ('isReply', String), ('pseudo_conversationId', Int64), ('pseudo_inReplyToUsername', String), ('pseudo_author_userName', Int64), ('quoted_pseudo_id', String), ('author_isBlueVerified', String)])


## 步骤 2: 数据清洗与类型规范化

In [3]:
# 收集数据并规范化布尔列
df = raw_lf.collect()
print(f"✅ 数据加载完成: {df.height:,} 行, {df.width} 列")

# 规范化布尔列
bool_columns = ['isReply', 'author_isBlueVerified']
df_cleaned = analysis.normalize_boolean_columns(df, bool_columns)
print(f"✅ 布尔列规范化完成: {bool_columns}")

# 转换时间字段为datetime类型
df_cleaned = df_cleaned.with_columns(
    pl.col('createdAt').str.to_datetime('%Y-%m-%d %H:%M:%S%#z').alias('createdAt')
)
print(f"✅ 时间字段转换完成")

✅ 数据加载完成: 508,954 行, 16 列
✅ 布尔列规范化完成: ['isReply', 'author_isBlueVerified']
✅ 时间字段转换完成


## 步骤 3: 添加事件相关时间字段（核心优化）

**关键时间点**:
- 枪击发生: 2025-09-10 (具体时间从数据推断)
- 数据窗口: 事件后72小时内的社交媒体反应

In [4]:
# 分析数据时间范围
print("\n📅 数据时间范围分析:")
min_time = df_cleaned['createdAt'].min()
max_time = df_cleaned['createdAt'].max()
print(f"  最早推文: {min_time}")
print(f"  最晚推文: {max_time}")
print(f"  数据跨度: {(max_time - min_time).total_seconds() / 3600:.1f} 小时")

# 推断枪击发生时间（假设为数据开始前的某个时间点）
# 根据数据最早时间推断：2025-09-11 23:55，枪击应该发生在9月10日
# 保守估计：9月10日下午（美国山地时间），约UTC时间9月10日晚上
SHOOTING_TIMESTAMP = datetime(2025, 9, 10, 20, 0, 0, tzinfo=timezone.utc)  # UTC时间
print(f"\n🎯 枪击事件时间（推断）: {SHOOTING_TIMESTAMP}")

# 添加事件相关字段（修复：Polars datetime 是微秒精度，需要 * 1_000_000）
df_with_event = df_cleaned.with_columns([
    # 距离枪击事件的时间差（小时）
    ((pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600).alias('event_time_delta_hours'),
    
    # 事件后时段标签
    pl.when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 6
    ).then(pl.lit('0-6h'))
    .when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 12
    ).then(pl.lit('6-12h'))
    .when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 24
    ).then(pl.lit('12-24h'))
    .when(
        (pl.col('createdAt').cast(pl.Int64) - pl.lit(int(SHOOTING_TIMESTAMP.timestamp() * 1_000_000))) / 1_000_000 / 3600 < 48
    ).then(pl.lit('24-48h'))
    .otherwise(pl.lit('48-72h'))
    .alias('time_window')
])

print(f"\n✅ 事件时间字段添加完成")
print(f"\n时段分布:")
print(df_with_event.group_by('time_window').agg(pl.len().alias('count')).sort('time_window'))


📅 数据时间范围分析:
  最早推文: 2025-09-11 23:55:56+00:00
  最晚推文: 2025-09-13 00:12:32+00:00
  数据跨度: 24.3 小时

🎯 枪击事件时间（推断）: 2025-09-10 20:00:00+00:00

✅ 事件时间字段添加完成

时段分布:
shape: (2, 2)
┌─────────────┬────────┐
│ time_window ┆ count  │
│ ---         ┆ ---    │
│ str         ┆ u32    │
╞═════════════╪════════╡
│ 24-48h      ┆ 415075 │
│ 48-72h      ┆ 93879  │
└─────────────┴────────┘


## 步骤 4: 数据质量检查

In [5]:
# 缺失值检查
key_cols = ['pseudo_id', 'pseudo_author_userName', 'createdAt', 'text']
missing_stats = profiling.missingness_summary(df_with_event, key_cols)
print("📊 缺失值统计 (top 5):")
print(missing_stats.head(5))

# 重复检查
dupes = profiling.duplicate_check(df_with_event, ['pseudo_id'])
print(f"\n🔍 重复推文数: {dupes.height}")

📊 缺失值统计 (top 5):
shape: (5, 4)
┌──────────────┬────────────┬────────────┬────────┐
│ column       ┆ null_count ┆ null_ratio ┆ is_key │
│ ---          ┆ ---        ┆ ---        ┆ ---    │
│ str          ┆ i64        ┆ f64        ┆ bool   │
╞══════════════╪════════════╪════════════╪════════╡
│ pseudo_id    ┆ 0          ┆ 0.0        ┆ true   │
│ text         ┆ 0          ┆ 0.0        ┆ true   │
│ retweetCount ┆ 0          ┆ 0.0        ┆ false  │
│ replyCount   ┆ 0          ┆ 0.0        ┆ false  │
│ likeCount    ┆ 0          ┆ 0.0        ┆ false  │
└──────────────┴────────────┴────────────┴────────┘

🔍 重复推文数: 50


## 步骤 5: 合并作者元数据

In [6]:
# 加载作者信息
authors_df = io.read_well_known_authors()
print(f"📋 作者元数据: {authors_df.height} 位作者")

# 类型转换：将 pseudo_author_userName 转为字符串以匹配 author_userName
df_with_str_author = df_with_event.with_columns(
    pl.col('pseudo_author_userName').cast(pl.Utf8).alias('pseudo_author_userName')
)

# 合并推文和作者数据（基于用户名）
df_with_authors = df_with_str_author.join(
    authors_df, 
    left_on='pseudo_author_userName', 
    right_on='author_userName',
    how='left'
)
print(f"✅ 数据合并完成: {df_with_authors.height:,} 行, {df_with_authors.width} 列")

📋 作者元数据: 4242 位作者
✅ 数据合并完成: 508,954 行, 25 列


## 步骤 6: 写入 Parquet 缓存

In [7]:
# 写入 parquet 文件供后续分析使用
output_path = Path("parquet/tweets_enriched.parquet")
io.materialize_parquet(df_with_authors.lazy(), output_path)

print(f"✅ Parquet 文件已生成: {output_path}")
print(f"📁 文件大小: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

print(f"\n新增字段:")
print(f"  - event_time_delta_hours: 距枪击事件的小时数")
print(f"  - time_window: 时段标签（0-6h, 6-12h, 12-24h, 24-48h, 48-72h）")

# 验证新字段
print(f"\n📊 事件时间字段示例:")
print(df_with_authors.select(['createdAt', 'event_time_delta_hours', 'time_window', 'text']).head(3))

✅ Parquet 文件已生成: parquet/tweets_enriched.parquet
📁 文件大小: 58.37 MB

新增字段:
  - event_time_delta_hours: 距枪击事件的小时数
  - time_window: 时段标签（0-6h, 6-12h, 12-24h, 24-48h, 48-72h）

📊 事件时间字段示例:
shape: (3, 4)
┌─────────────────────────┬────────────────────────┬─────────────┬─────────────────────────────────┐
│ createdAt               ┆ event_time_delta_hours ┆ time_window ┆ text                            │
│ ---                     ┆ ---                    ┆ ---         ┆ ---                             │
│ datetime[μs, UTC]       ┆ f64                    ┆ str         ┆ str                             │
╞═════════════════════════╪════════════════════════╪═════════════╪═════════════════════════════════╡
│ 2025-09-13 00:12:32 UTC ┆ 52.208889              ┆ 48-72h      ┆ CLEARLY WHY ROBINSO. KILLED CH… │
│ 2025-09-13 00:12:32 UTC ┆ 52.208889              ┆ 48-72h      ┆ @695242549121979 Charlie Kirk … │
│ 2025-09-13 00:12:32 UTC ┆ 52.208889              ┆ 48-72h      ┆ @396187379099632 @57679415126

## ✅ 数据接入完成！

**新增核心字段**:
- `event_time_delta_hours`: 距枪击事件的时间差（小时）
- `time_window`: 事件后时段标签

**下一步**: 运行 `03_content_semantics.ipynb` 进行情感与叙事分析